In [1]:
import tensorflow as tf
from tensorflow import keras

print(tf.__version__, keras.__version__)

2.3.0 2.4.0


## Setting up log location

To use tensorboard, you must modify your program to output files for visualisation to special binary log files called *event files*. Each binary data record is called *summary*. 

Tensorboard service will monitor root log directory and automatically update visualisations

In [21]:
import os
import time

root_logdir = os.path.join(os.curdir, "tf_logs")

def run_logdir():
    root_logdir = os.path.join(os.curdir, "tf_logs")
    run_id = time.strftime("%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)


print(run_logdir())

.\tf_logs\2020_09_22-20_49_44


### Downloading Fashion mnist dataroot_logdir

In [13]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

print(X_train_full.shape, X_test.shape)

(60000, 28, 28) (10000, 28, 28)


### Create Validation Data Set

Also in addition to that do a very basic scaling (ie. dividing data by 255)

In [14]:
X_train, X_valid  = X_train_full[5000:] / 255.0, X_train_full[:5000] / 255.0
y_train, y_valid  = y_train_full[5000:], y_train_full[:5000]

print(X_train.shape, X_valid.shape)

(55000, 28, 28) (5000, 28, 28)


In [15]:
class_names = ["T-shirt/pop", "Trouser", "Pullover", "Dresss", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

## Building Keras Model

In [26]:
tf.keras.backend.clear_session() # remove all previous weights
tf.random.set_seed(1234) # set same seeds

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation = "relu"))
model.add(keras.layers.Dense(300, activation = "relu"))
model.add(keras.layers.Dense(100, activation = "relu"))
model.add(keras.layers.Dense(10, activation = "softmax"))

In [27]:
# both statements are the same
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Define Callback Functions for TensorBoard

In [28]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir())

In [29]:
NUM_EPOCHS = 200
history = model.fit(X_train, y_train, epochs=NUM_EPOCHS, 
                    validation_data=(X_valid, y_valid), 
                    verbose = False, callbacks = [tensorboard_cb])
#history = model.fit(X_train_full, y_train_full, epochs=NUM_EPOCHS, validation_split = 0.2)

### Loading from Jupyter

```python
%load_ext tensorboard
%tensorboard --logdir=./tf_logs --port=6006
```

### Lower Level API

Additionally, TensorFlow offers a lower-level API in the **tf.summary** package, where we can log scalars, histograms, images, audio, and text. All of which can be visualised with TensorBoard

In [31]:
import numpy as np

test_logdir = "./tf_log_file_write_example"
writer = tf.summary.create_file_writer(test_logdir)

with writer.as_default():
    for step in range(1, 1000 + 1):
        tf.summary.scalar("my_scalar", np.sin(step/10), step = step)
        data = (np.random.randn(100) + 2) * step / 100 #some random data
        tf.summary.histogram("my_hist", data, buckets = 50, step = step)
        images = np.random.rand(2, 32, 32, 3) # random 32x32 RGB image
        tf.summary.image("my_images", images *step/1000, step = step)
        texts = [f"The step is {step}", f"it's square is {step**2}"]
        tf.summary.text("my_text", texts, step = step)
        sine_wave = tf.math.sin(tf.range(12000) / 48000 * 2 *np.pi * step)
        audio = tf.reshape(tf.cast(sine_wave, tf.float32), [1, -1, 1])
        tf.summary.audio("my_audio", audio, sample_rate = 48000, step = step)